In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.2 MB/s eta 0:00:00


In [13]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.205 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.4/107.7 GB disk)


In [23]:
!cp -r /content/drive/MyDrive/TL_D/v8_format/* .

In [24]:
# !rsync -a /content/drive/MyDrive/TL_D/v8_format/* .

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=data.yaml epochs=50 imgsz=640 batch=8 project=/content/drive/MyDrive/ball_output name=Ball_Detection

Ultralytics 8.3.205 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=Ball_Detection3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pre

In [7]:
# !yolo task=detect mode=train model=yolov8s.pt data=data.yaml epochs=50 imgsz=640 batch=8 project=/content/drive/MyDrive/ball_output name=Ball_Detection_1 optimizer='AdamW' close_mosaic=10 cos_lr=True degrees: 0.2 shear: 0.2 perspective: 0.01 mixup: 0.3 warmup_epochs=1.0

In [8]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/ball_output/Ball_Detection/weights/best.pt conf=0.55 source=/content/drive/MyDrive/v8_format/images/val/002af65597d93000.jpg save=true

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/cfg/__init__.py", line 960, in entrypoint
    model = YOLO(model, task=task)
            ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/models/yolo/model.py", line 83, in __init__
    super().__init__(model=model, task=task, verbose=verbose)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/model.py", line 153, in __init__
    self._load(model, task=task)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/model.py", line 297, in _load
    self.model, self.ckpt = load_checkpoint(weights)
                            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py", line 1501, in load_checkpoint
    ckpt, weight = torch_safe_load(weight)  # load ckpt
                   ^^^^^^^^^

In [ ]:
!pip install supervision

In [ ]:
import cv2
from ultralytics import YOLO
import supervision as sv
import numpy as np

# --- Configuration ---
# Note: Make sure these paths are correct for your Colab environment.
input_video_path = "/content/drive/MyDrive/29_sept_IITD/foot_ball.mp4"
output_video_path = "/content/drive/MyDrive/29_sept_IITD/detection_output.mp4"
model_path = "/content/drive/MyDrive/ball_output/Ball_Detection/weights/best.pt"

# --- Video Capture ---
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print(f"Error: Could not open video file at {input_video_path}")
else:
    # --- Get Video Properties ---
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # --- Video Writer ---
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # --- Model and Annotators ---
    model = YOLO(model_path)

    # Create a BoxAnnotator for drawing boxes ONLY
    box_annotator = sv.BoxAnnotator(
        thickness=2
    )

    # Create a LabelAnnotator for drawing text labels ONLY
    label_annotator = sv.LabelAnnotator(
        text_thickness=2,
        text_scale=1
    )

    print("🚀 Starting video processing...")

    # --- Main Loop ---
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        result = model(frame, agnostic_nms=True)[0]
        detections = sv.Detections.from_ultralytics(result)

        # CORRECTED LINE: Access class_id and confidence directly from detections
        labels = [
            f"{model.model.names[class_id]} {confidence:0.2f}"
            for class_id, confidence
            in zip(detections.class_id, detections.confidence)
        ]

        # --- Two-step annotation process ---
        # 1. Annotate the frame with bounding boxes
        frame = box_annotator.annotate(
            scene=frame,
            detections=detections
        )

        # 2. Annotate the frame with labels on top of the boxes
        frame = label_annotator.annotate(
            scene=frame,
            detections=detections,
            labels=labels
        )

        out.write(frame)

    print(f"✅ Processing complete. Video saved to {output_video_path}")

    # --- Cleanup ---
    cap.release()
    out.release()
    cv2.destroyAllWindows()